<a href="https://colab.research.google.com/github/Pushkar-05/Opencv_augmented_Reality/blob/main/NonLinearRL_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from numpy import matrix

import quadrotor

# Part 4 Task 2

In [ ]:
# we can get its mass, half length (r), gravity constant
print(f'm is {quadrotor.MASS}')
print(f'r is {quadrotor.LENGTH}')
print(f'I is {quadrotor.INERTIA}')
print(f'g is {quadrotor.GRAVITY}')

# we can also get the integration step used in the simulation
print(f'dt is {quadrotor.DELTA_T}')

# we can get the size of its state and control vector
print(f'number of states {quadrotor.NUMBER_STATES} and number of controls {quadrotor.NUMBER_CONTROLS}')
print('the states are indexed as follows: x, vx, y, vy, theta, omega')

m is 0.6
r is 0.2
I is 0.15
g is 9.81
dt is 0.01
number of states 6 and number of controls 2
the states are indexed as follows: x, vx, y, vy, theta, omega


In [ ]:
def M(A,B):
    return np.matmul(A,B)

def inv(A):
    return np.linalg.inv(A)

def T(A):
    return np.matrix.transpose(A)

def getKcurr(B,P,R,A):
    #print(f'shape is: {B.shape}')
    C = np.matmul(np.matmul(matrix.transpose(B),P),B)
    D = np.matmul(np.matmul(matrix.transpose(B),P),A)
    #print(f'c = {C} \n d = {D}')
    return -1*np.matmul(np.linalg.inv(C+R),D)


def getPcurr(Q,A,P,B,K):
    C = np.matmul(np.matmul(matrix.transpose(A),P),A)
    D = np.matmul(np.matmul(matrix.transpose(A),P),np.matmul(B,K))
    return Q+C+D


In [ ]:

def calc_derivs(x,u):
    DOF = 3
    eps = 1e-3  # finite difference epsilon
        #----------- compute xdot_x and xdot_u using finite differences --------
        # NOTE: here each different run is in its own column
    A = np.zeros([x.shape[0],x.shape[0]])
    inc = np.zeros([x.shape[0],])
    for i in range(x.shape[0]):
        inc[i] = 1
        x1 = x+eps*inc
        x2 = x-eps*inc
        f1 = quadrotor.get_next_state(x1, u)
        f2 = quadrotor.get_next_state(x2, u)
        A[:,i] = (f1-f2)/(2*eps)
        inc[i]=0

    B = np.zeros([x.shape[0],u.shape[0]])
    inc = np.zeros([u.shape[0],])
    for i in range(u.shape[0]):
        inc[i] = 1
        u1 = u+eps*inc
        u2 = u-eps*inc
        f1 = quadrotor.get_next_state(x, u1)
        f2 = quadrotor.get_next_state(x, u2)
        B[:,i] = (f1-f2)/(2*eps)
        inc[i]=0

    return A,B




x = np.zeros([quadrotor.NUMBER_STATES,])
u = np.zeros([2,])
u[0] = (quadrotor.MASS * quadrotor.GRAVITY)/2
u[1] = u[0]

A,B = calc_derivs(x,u)
print(A,B)


[[ 1.          0.01        0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.         -0.09809998  0.        ]
 [ 0.          0.          1.          0.01        0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.01      ]
 [ 0.          0.          0.          0.          0.          1.        ]] [[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.01666667  0.01666667]
 [ 0.          0.        ]
 [ 0.01333333 -0.01333333]]


In [ ]:

def get_linearization():
    dt = quadrotor.DELTA_T
    m = quadrotor.MASS
    I = quadrotor.INERTIA
    r = quadrotor.LENGTH
    g = quadrotor.GRAVITY

    A = np.array([[1,dt,0,0,0,0],
                  [0,1,0,0,-g*dt,0],
                 [0,0,1,dt,0,0],
                 [0,0,0,1,0,0],
                 [0,0,0,0,1,dt],
                 [0,0,0,0,0,1]]);
    B = np.array([[0,0],[0,0],[0,0],[dt/m,dt/m],[0,0],[r*dt/I,-r*dt/I]])
    return A,B

import math

def getKtracking(R,B,P,p_track,r):
    #print(f'shape is: {B.shape}')
    #-1*inv(B'PB+R)B'p
    #print(f'R = {R} \n B= {B} \n P= {P}\n p_track={p_track} \n r= {r}')
    C = np.matmul(np.matmul(matrix.transpose(B),P),B)
    D = np.matmul(matrix.transpose(B),p_track)
    #print(f'c = {C} \n d = {D}')
    return -1*np.matmul(np.linalg.inv(C+R),D+r)


def getPtracking(q_track,p_track,A,P,B,k_track):
    #q_track+A'p_track+A'*P*B*k_track
    C = np.matmul(matrix.transpose(A),p_track)
    D = np.matmul(np.matmul(matrix.transpose(A),P),np.matmul(B,k_track))
    return q_track+C+D


In [ ]:
#Non linear trajectory
import scipy.linalg as sp_linalg



def get_state_der(current_time):
    w = 0.007
    state_desired = np.zeros([quadrotor.NUMBER_STATES,])
    if current_time<450:
        return state_desired
    if current_time<550:
        state_desired[0] = 3 #math.cos(w*(current_time))
        state_desired[1] = 0 #-w*math.sin(w*(current_time))
        state_desired[2] = 3 #math.sin(w*(current_time))
        state_desired[3] = 0 #w*math.cos(w*(current_time))
        state_desired[4] = np.pi
        return state_desired
    else:
        state_desired[4] = 2*np.pi


#     state_desired[5] = 10
    return state_desired


In [ ]:
#iterative LQR
Q_ = np.diag(np.full(6,100))
R_  = np.diag(np.full(2,1))

Q_[1,1] = 5
Q_[3,3] = 5
Q_[5,5] = 5
Q_task = 20*Q_
z0 = np.zeros([quadrotor.NUMBER_STATES,])
print(Q_)
print(R_)
def getCost(state,u):
    cost =0
    for i in range(450):
        cost+= state[i,:].T @(Q_@state[i,:])+u[i,:].T@(R_@u[i,:])

    for i in range(450,550):
        cost+= (state[i,:]-get_state_der(i)) @(Q_task@(state[i,:]-get_state_der(i)))+u[i,:].T@(R_@u[i,:])

    for i in range(550,1000):
        cost+= (state[i,:]-get_state_der(i)) @(Q_@(state[i,:]-get_state_der(i)))+u[i,:].T@(R_@u[i,:])
#     for i in range(1000):
#          cost+= (state[i,:]-get_state_der(i)) @(Q_@(state[i,:]-get_state_der(i)))+u[i,:].T@(R_@u[i,:])
    cost+=(state[1000,:]-get_state_der(1000)) @(Q_@(state[1000,:]-get_state_der(1000)))
    return cost

def getCurrTrajectroyPath(x0,u):
    x = np.zeros([1001,6])
    x[0] = x0
    old_state = x0
    for i in range(1,u.shape[0]):
        curr_state = quadrotor.get_next_state(old_state,u[i,:])
        x[i] = (curr_state)
        old_state = curr_state
    return x




def getNewKs(x_star,u_star):
    N = x_star.shape[0]-1
    P_prev,p_track_prev,_,_ = calculateDerivativeCost(x_star,u_star,N)
    K_arrays = []
    k_track_arrays = []
    for i in range(N):
        n = N-i-1
        A,B = calc_derivs(x_star[n,:],u_star[n,:])
        Q,q,R,r = calculateDerivativeCost(x_star,u_star,n)
        K_curr = getKcurr(B,P_prev,R,A)
        P_curr = getPcurr(Q,A,P_prev,B,K_curr)
        k_track_curr = getKtracking(R,B,P_prev,p_track_prev,r)
        p_track_curr = getPtracking(q,p_track_prev,A,P_prev,B,k_track_curr)
        P_prev = P_curr
        p_track_prev = p_track_curr
        K_arrays.append(K_curr)
        k_track_arrays.append(k_track_curr)
    K_arrays.reverse()
    k_track_arrays.reverse()
    return K_arrays,k_track_arrays


def getNewControls(x_star,u_star,alpha):
    #K,k = getNewKs(x_star,u_star)
    newU = np.zeros(u_star.shape)
    prev_state = x_star[0]

    for i in range(u_star.shape[0]):
        newU[i,:] = u_star[i,:] + K[i]@(prev_state - x_star[i,:])
        newU[i,:] += alpha*k[i].reshape(2,)
        prev_state = quadrotor.get_next_state(prev_state,newU[i,:])
    return newU




[[100   0   0   0   0   0]
 [  0   5   0   0   0   0]
 [  0   0 100   0   0   0]
 [  0   0   0   5   0   0]
 [  0   0   0   0 100   0]
 [  0   0   0   0   0   5]]
[[1 0]
 [0 1]]


In [ ]:
# def calculateDerivativeCost(x_star,u_star,timestep):

#     f = getCost(x_star,u_star)
#     eps = 1e-5
#     Q = np.zeros([6,6])
#     q = np.zeros([6,1])
#     R = np.zeros([2,2])
#     r = np.zeros([2,1])
#     inc = np.zeros([6,])
#     for i in range(x_star[timestep,:].shape[0]):
#         inc[i] = 1
#         x_star[timestep,:] +=eps*inc
#         f1 = getCost(x_star,u_star)
#         x_star[timestep,:] -=2*eps*inc
#         f2 = getCost(x_star,u_star)
#         #print(f'f1 = {f1} f2 = {f2} f3 = {f}')
#         Q[i,i] = (f1+f2-2*f)/(eps**2)
#         q[i,0] = (f1-f2)/(2*eps)
#         x_star[timestep,:] += eps*inc
#         inc[i] = 0

#     inc = np.zeros([2,])
#     if(timestep != 1000 ):
#         for i in range(u_star[timestep,:].shape[0]):
#             inc[i] = 1
#             u_star[timestep,:] +=eps*inc
#             f1 = getCost(x_star,u_star)
#             u_star[timestep,:] -=2*eps*inc
#             f2 = getCost(x_star,u_star)
#             R[i,i] = (f1+f2-2*f)/(eps**2)
#             r[i,0] = (f1-f2)/(2*eps)
#             u_star[timestep,:] += eps*inc
#             inc[i] = 0



#     return Q,q,R,r


def calculateDerivativeCost(x_star,u_star,timestep):

    Q = np.zeros([6,6])
    q = np.zeros([6,1])
    R = np.zeros([2,2])
    r = np.zeros([2,1])
    if(timestep<450 or timestep>=550):
        Q = 2*Q_
        q = (2*(x_star[timestep,:]-get_state_der(timestep)).T@Q_).T
        if(timestep!=1000):
            R = 2*R_
            r = (2*(u_star[timestep,:]).T@R_).T
        return Q,q,R,r
    else:
        Q = 2*Q_task
        R = 2*R_
        q = (2*(x_star[timestep,:]-get_state_der(timestep)).T@(Q_task)).T
        r = (2*(u_star[timestep,:]).T@R_).T
        return Q,q,R,r

#Q,q,R,r = calculateDerivativeCost(new_state,new_u,500)
#print(Q,q,R,r)

In [ ]:


#code to test calculateDerivativeCost
# Q,q,R,r = calculateDerivativeCost(new_state,new_u,0)

# incx = np.ones([6,])*1e-5
# incu = np.ones([2,])*1e-5
# new_state_delta = new_state.copy()
# new_u_delta =  new_u.copy()
# new_state_delta[0,:]+=incx
# new_u_delta[0,:]+=incu
# f1 = getCost(new_state_delta,new_u_delta)
# f2 = getCost(new_state,new_u)+q.T@incx+(incx.T@(Q@incx))/2+r.T@incu+(incu.T@(R@incu))/2
# f3 = getCost(new_state,new_u)
def controller(state,i):
    return old_u[i]

horizon_length = 1000


In [ ]:
converge = False
old_cost= 1e15

iteration = 0
old_u = (quadrotor.MASS * quadrotor.GRAVITY)/2* np.ones([1000,2])
while not converge:
    print(f'old cost = {old_cost}')
    iteration += 1
    horizon_length = 1000
    old_state = getCurrTrajectroyPath(z0,old_u)
    K,k = getNewKs(old_state,old_u)
    alpha=1
    new_u = old_state.copy()
    new_state = old_state.copy()
    cost = old_cost+1
    while(cost>old_cost and alpha>1e-4):
        print(f'trying alpha {alpha}')
        new_u = getNewControls(old_state,old_u,alpha)
        new_state = getCurrTrajectroyPath(z0,new_u)
        cost = getCost(new_state,new_u)
        alpha/=2

    if(cost<old_cost):
        old_u = new_u.copy()
        old_cost = cost
    else:
        converge = True


old cost = 1000000000000000.0
trying alpha 1
old cost = 5481762.926187336
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 4119790.598619005
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 3442469.333460556
trying alpha 1
trying alpha 0.5
old cost = 2547118.6880450747
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 2261649.5748643475
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 1986241.7412097538
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 1727481.11938972
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 1525480.3364326092
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 1329031.0130309842
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 1164790.1664842805
trying alpha 1
trying alpha 0.5
old cost = 1089143.0865353362
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 914799.3902818564
trying alpha 1
trying alpha 0.5
trying alpha 0.25
old cost = 767152.0359570956
trying a

In [ ]:



t, state, u = quadrotor.simulate(z0, controller, horizon_length, disturbance = False)




In [ ]:
print(state[:,500])

[ 2.91287713  0.01025122  2.94303428 -0.01786878  3.2804902   0.21049812]


In [ ]:
quadrotor.animate_robot(state, u)

Your browser does not support the video tag.